In [9]:
import nltk
import numpy as np
import collections
import math

nltk.download('universal_tagset')
nltk.download('brown')
train = nltk.corpus.brown.tagged_sents(tagset='universal')[:10000]

[nltk_data] Downloading package universal_tagset to C:\Users\Aniket
[nltk_data]     Dalvi\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package brown to C:\Users\Aniket
[nltk_data]     Dalvi\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [10]:
# creating initial distribution dictionary
oov_threshold = 2
init_dist_dict = {}
for sentence in train:
    word = sentence[0][1]
    if (word in init_dist_dict):
        init_dist_dict[word] = init_dist_dict[word] + 1
    else:
        init_dist_dict[word] = 1

init_dist = {}
total_sent = len(train)
for key, value in init_dist_dict.items():
    init_dist[key] = (value + 0.001)/(total_sent + 0.001)

In [11]:
# pre-processsing pos tags in training data to create pi array and index map for pos
pos_list = []
pi = []
for sentence in train:
    for word in sentence:
        pos_list.append(word[1])

uniq_pos = list(set(pos_list))
pos_index_map = {}
for i in range(len(uniq_pos)):
    pos_index_map[uniq_pos[i]] = i
    if(uniq_pos[i] in init_dist):
        pi.append(init_dist[uniq_pos[i]])
    else:
        pi.append(0.0)

In [12]:
# creating transition matrix
transition_counts = [ [0]*len(uniq_pos) for i in range(len(uniq_pos))]

for sentence in train:
    for j in range(len(sentence)-1):
        x = pos_index_map[sentence[j][1]]
        y = pos_index_map[sentence[j+1][1]]
        transition_counts[x][y] = transition_counts[x][y] + 1

transition_mat = [ [0]*len(uniq_pos) for i in range(len(uniq_pos))]

for x in range(len(uniq_pos)):
    for y in range(len(uniq_pos)):
        transition_mat[x][y] = (transition_counts[x][y] + 0.001)/(sum(transition_counts[x]) + 0.001)

In [13]:
# pre-processsing words in training data to create index map for words
all_words = []
for sentence in train:
    for words in sentence:
        all_words.append(words[0].lower())

uniq_words_oov = []
words_index_map = {}
word_freq = collections.Counter(all_words)
for key in word_freq.keys():
    if (word_freq[key] > oov_threshold):
        uniq_words_oov.append(key)
uniq_words_oov.append('oov')
for i in range(len(uniq_words_oov)):
    words_index_map[uniq_words_oov[i]] = i

In [14]:
# creating observation matrix
observation_counts = [ [0]*len(uniq_words_oov) for i in range(len(uniq_pos))]
for sentence in train:
    for words in sentence:
        x = pos_index_map[words[1]]
        if(words[0].lower() not in uniq_words_oov):
            y = words_index_map['oov']
        else:
            y = words_index_map[words[0].lower()]
        observation_counts[x][y] = observation_counts[x][y] + 1

observation_mat = [ [0]*len(uniq_words_oov) for i in range(len(uniq_pos))]
for x in range(len(uniq_pos)):
    for y in range(len(uniq_words_oov)):
        observation_mat[x][y] = (observation_counts[x][y] + 0.001)/(sum(observation_counts[x]) + 0.001)

In [15]:
# implementing the viterbi algorithm
def viterbi(obs, pi, a, b):
    dp_matrix = [ [0]*len(obs) for i in range(len(uniq_pos))]
    state_matrix = [ [0]*len(obs) for i in range(len(uniq_pos))]
    for x in range(len(uniq_pos)):
        for y in range(len(obs)):
            if (y==0):
                dp_matrix[x][y] = np.exp(np.log(pi[x]) + np.log(b[x][obs[y]]))
                state_matrix[x][y] = -1

    for y in range(1, len(obs)):
        for x in range(len(uniq_pos)):
            max_prob = float('-inf')
            backtrack_state = -1
            for z in range(len(uniq_pos)):
                prob = np.exp(np.log(dp_matrix[z][y-1]) + np.log(a[z][x]) + np.log(b[x][obs[y]]))
                if (prob>max_prob):
                    max_prob = prob
                    backtrack_state = z
            dp_matrix[x][y] = (max_prob)
            state_matrix[x][y] = backtrack_state
    fin_max = float('-inf')
    fin_state = -1
    for j in range(len(uniq_pos)):
        fin_prob = dp_matrix[j][len(obs)-1]
        if(fin_prob>fin_max):
            fin_max = fin_prob
            fin_state = j
    curr_best = fin_state
    best_states = [curr_best]
    for j in range(len(obs)-1,0,-1):
        curr_best = state_matrix[curr_best][j]
        best_states = [curr_best] + best_states

    return best_states

In [16]:
# testing pos tagging on sample data using viterbi
test = nltk.corpus.brown.tagged_sents(tagset='universal')[10150:10155]
transition_mat = np.array(transition_mat)
observation_mat = np.array(observation_mat)
test_sentences = []
for sentence in test:
    test_sentence = []
    for words in sentence:
        if(words[0].lower() not in uniq_words_oov):
            test_sentence.append(words_index_map['oov'])
        else:
            test_sentence.append(words_index_map[words[0].lower()])
    test_sentences.append(test_sentence)

index_pos_map = {y:x for x,y in pos_index_map.items()}
predicted_states = []
for test_sentence in test_sentences:
    predicted_states.append([index_pos_map[x] for x in viterbi(test_sentence, pi, transition_mat, observation_mat)])

index_test = 0
for raw_test in test:
    print('Test Case:')
    print(raw_test)
    print('Predicted:')
    print(predicted_states[index_test])
    index_test += 1

Test Case:
[('Those', 'DET'), ('coming', 'VERB'), ('from', 'ADP'), ('other', 'ADJ'), ('denominations', 'NOUN'), ('will', 'VERB'), ('welcome', 'VERB'), ('the', 'DET'), ('opportunity', 'NOUN'), ('to', 'PRT'), ('become', 'VERB'), ('informed', 'VERB'), ('.', '.')]
Predicted:
['DET', 'VERB', 'ADP', 'ADJ', 'NOUN', 'VERB', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB', 'VERB', '.']
Test Case:
[('The', 'DET'), ('preparatory', 'ADJ'), ('class', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('introductory', 'ADJ'), ('face-to-face', 'ADJ'), ('group', 'NOUN'), ('in', 'ADP'), ('which', 'DET'), ('new', 'ADJ'), ('members', 'NOUN'), ('become', 'VERB'), ('acquainted', 'VERB'), ('with', 'ADP'), ('one', 'NUM'), ('another', 'DET'), ('.', '.')]
Predicted:
['DET', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'ADP', 'NUM', 'DET', '.']
Test Case:
[('It', 'PRON'), ('provides', 'VERB'), ('a', 'DET'), ('natural', 'ADJ'), ('transition', 'NOUN'), ('into', 'ADP'), ('the', 'DET'